In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
train_images.shape

(60000, 28, 28)

In [4]:
test_images.shape

(10000, 28, 28)

In [5]:
train_images[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  

In [6]:
train_labels[0]

5

Each image is a 28 * 28 greyscale image, each pixel having a value between 0 and 255. We will do the following changes to the images so that we can use them in a dense layer:-
1. Reshape the images from 28 X 28 to 784 X 1.
2. Normalize the values to range (0,1).

In [7]:
x_train = train_images.reshape((len(train_images), 28 * 28))
x_train = x_train.astype("float32") / 255

x_test = test_images.reshape((len(test_images), 28 * 28))
x_test = x_test.astype("float32") / 255

In [8]:
x_train.shape

(60000, 784)

In [9]:
x_test.shape

(10000, 784)

In [10]:
y_train = train_labels.astype("float32")
y_test = test_labels.astype("float32")

In [33]:
class RootMeanSquaredError(tf.metrics.Metric):
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(name="total_samples", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.one_hot(tf.cast(y_true, "int32"), depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(tf.cast(num_samples, "float32"))

    def result(self):
        return tf.sqrt(self.mse_sum / self.total_samples)

    def reset_state(self):
        self.mse_sum.assign(0.)
        self.total_samples.assign(0.)

In [34]:
model = keras.Sequential([
                          keras.layers.Dense(128, activation="relu"),
                          keras.layers.Dense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy", RootMeanSquaredError()])

In [35]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10)

Epoch 1/10
469/469 [==============================] - 3s 5ms/step - loss: 0.3332 - accuracy: 0.9072 - rmse: 14.2098
Epoch 2/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1596 - accuracy: 0.9540 - rmse: 14.7101
Epoch 3/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1156 - accuracy: 0.9663 - rmse: 14.8254
Epoch 4/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0904 - accuracy: 0.9740 - rmse: 14.8842
Epoch 5/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0735 - accuracy: 0.9785 - rmse: 14.9254
Epoch 6/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0621 - accuracy: 0.9818 - rmse: 14.9538
Epoch 7/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0530 - accuracy: 0.9844 - rmse: 14.9746
Epoch 8/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0454 - accuracy: 0.9869 - rmse: 14.9898
Epoch 9/10
469/469 [==============================] - 2s 5ms/step - loss